In [1]:
import pandas as pd

# Duplicate Analysis


In [2]:
PATH_PREFIX = "data/model"
PATH_PREFIX_REMOTE = "s3://aclu-model-policy-comments"

In [3]:
def fetch_table(table_name, fetch_remote=True):
    return pd.read_csv(
        f"{PATH_PREFIX_REMOTE if fetch_remote else PATH_PREFIX}/{table_name}.csv",
        lineterminator="\n",
    )

In [13]:
comments = fetch_table("comments")
classifications = fetch_table("classifications")
dupe_groups = fetch_table("dupe_groups")
dupe_group_comments = fetch_table("dupe_group_comments")

comments.head()

,id,url,label,title,content,html_raw,created_at,author,full_text
0,203047,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Transgender,I am strongly opposed to any law that would ...,<!DOCTYPE html> <p>I am strongly opposed to ...,2022-10-27 03:07:00,Frank McCullough,Transgender I am strongly opposed to any law...
1,203046,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,"OPPOSE - this will cause actual, real, death o...","These kids are valid in their identities, ta...",<!DOCTYPE html> <p>These kids are valid in t...,2022-10-27 01:55:00,an APS student,"OPPOSE - this will cause actual, real, death o..."
2,203045,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,"Consequences of Youngkin’s Policy Dire, Immediate",Gov. Youngkin's model policies regarding tra...,<!DOCTYPE html> <p>Gov. Youngkin's model pol...,2022-10-27 00:06:00,"Martha Molinaro, George Mason University","Consequences of Youngkin’s Policy Dire, Immedi..."
3,203044,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Strongly Oppose,I strongly oppose the non-evidence based pol...,<!DOCTYPE html> <p>I <strong>strongly oppose...,2022-10-27 00:04:00,Anonymous (203044),Strongly Oppose I strongly oppose the non-ev...
4,203041,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Opposed,"""Parents"" put a private equity sellout in ch...","<!DOCTYPE html> <p>""Parents"" put a private e...",2022-10-27 00:02:00,Anonymous (203041),"Opposed ""Parents"" put a private equity sello..."


In [72]:
comments["created_at"] = pd.to_datetime(comments["created_at"])

In [15]:
import duckdb

In [24]:
query = """
    SELECT
        dupe_groups.id,
        COUNT(*) AS count_members,
    FROM dupe_groups
    LEFT JOIN dupe_group_comments
        ON dupe_groups.id = dupe_group_comments.group_id
    GROUP BY dupe_groups.id
"""
duckdb.query(query).to_df().head()

,id,count_members
0,0,4419
1,1,4227
2,2,2740
3,3,1540
4,4,1363


In [42]:
query = """
    SELECT
        dupe_groups.id AS group_id,
        comments.id AS comment_id,
        comments.created_at,
        LEAD(created_at) OVER(PARTITION BY dupe_groups.id ORDER BY created_at) AS next_comment_created_at,
        LEAD(created_at) OVER(PARTITION BY dupe_groups.id ORDER BY created_at) - comments.created_at AS created_timedelta,

    FROM dupe_groups
    LEFT JOIN dupe_group_comments
        ON dupe_groups.id = dupe_group_comments.group_id
    LEFT JOIN comments
        ON dupe_group_comments.comment_id = comments.id
    ORDER BY group_id, created_at
"""

duckdb.query(query).to_df()

,group_id,comment_id,created_at,next_comment_created_at,created_timedelta
0,0,163606,2022-09-28 20:01:00,2022-09-28 20:01:00,0 days 00:00:00
1,0,163609,2022-09-28 20:01:00,2022-09-28 20:01:00,0 days 00:00:00
2,0,163613,2022-09-28 20:01:00,2022-09-28 20:01:00,0 days 00:00:00
3,0,163616,2022-09-28 20:01:00,2022-09-28 20:01:00,0 days 00:00:00
4,0,163622,2022-09-28 20:01:00,2022-09-28 20:01:00,0 days 00:00:00
...,...,...,...,...,...
41783,1624,197557,2022-10-25 22:01:00,NaT,NaT
41784,1625,129123,2022-09-26 09:19:00,2022-09-26 09:19:00,0 days 00:00:00
41785,1625,129122,2022-09-26 09:19:00,NaT,NaT
41786,1626,129066,2022-09-26 09:06:00,2022-09-26 09:07:00,0 days 00:01:00


## Bot Groups


In [43]:
query = """
WITH group_deltas AS (
    SELECT
        dupe_groups.id AS group_id,
        dupe_groups.sample_comment_text AS sample_comment_text,
        comments.id AS comment_id,
        comments.created_at,
        LEAD(created_at) OVER(PARTITION BY dupe_groups.id ORDER BY created_at) AS next_comment_created_at,
        LEAD(created_at) OVER(PARTITION BY dupe_groups.id ORDER BY created_at) - comments.created_at AS created_timedelta,

    FROM dupe_groups
    LEFT JOIN dupe_group_comments
        ON dupe_groups.id = dupe_group_comments.group_id
    LEFT JOIN comments
        ON dupe_group_comments.comment_id = comments.id
)

SELECT
    group_id AS id,
    ANY_VALUE(sample_comment_text) AS sample_comment_text,
    COUNT(*) AS count_members,
    COUNT(*) FILTER(created_timedelta <= INTERVAL 2 MINUTE) AS count_members_less_than_2_minute_interval,

FROM group_deltas
GROUP BY group_id
ORDER BY group_id
"""

dupe_groups = duckdb.query(query).to_df()

dupe_groups

,id,sample_comment_text,count_members,count_members_less_than_2_minute_interval
0,0,Leave Trans Kids Alone This will hurt kids. ...,4419,4417
1,1,Strongly Oppose This will hurt kids. Don't b...,4227,4222
2,2,Youngkin Hates This will hurt kids. Don't be...,2740,2738
3,3,I fully oppose this. I fully oppose this.,1540,1459
4,4,Glenn Youngkin Is A Bot This will hurt kids....,1363,1362
...,...,...,...,...
1622,1622,Strongly SUPPORT! STRONGLY support this chan...,2,0
1623,1623,Transphobia NoVa schools have some of the mo...,2,1
1624,1624,100% Support!! Support this revised policyI...,2,0
1625,1625,Strongly oppose Strongly oppose your policie...,2,1


In [44]:
dupe_groups["less_than_2_min_ratio"] = (
    dupe_groups["count_members_less_than_2_minute_interval"]
    / dupe_groups["count_members"]
)

dupe_groups.head()

,id,sample_comment_text,count_members,count_members_less_than_2_minute_interval,less_than_2_min_ratio
0,0,Leave Trans Kids Alone This will hurt kids. ...,4419,4417,0.999547
1,1,Strongly Oppose This will hurt kids. Don't b...,4227,4222,0.998817
2,2,Youngkin Hates This will hurt kids. Don't be...,2740,2738,0.999270
3,3,I fully oppose this. I fully oppose this.,1540,1459,0.947403
4,4,Glenn Youngkin Is A Bot This will hurt kids....,1363,1362,0.999266


In [45]:
dupe_groups["is_bot"] = dupe_groups["less_than_2_min_ratio"] > 0.9

dupe_groups.head()

,id,sample_comment_text,count_members,count_members_less_than_2_minute_interval,less_than_2_min_ratio,is_bot
0,0,Leave Trans Kids Alone This will hurt kids. ...,4419,4417,0.999547,True
1,1,Strongly Oppose This will hurt kids. Don't b...,4227,4222,0.998817,True
2,2,Youngkin Hates This will hurt kids. Don't be...,2740,2738,0.999270,True
3,3,I fully oppose this. I fully oppose this.,1540,1459,0.947403,True
4,4,Glenn Youngkin Is A Bot This will hurt kids....,1363,1362,0.999266,True


Sentiment Counts


In [50]:
query = """
    WITH group_counts AS (
        SELECT
            dupe_groups.id,
            COUNT(*) FILTER(classifications.sentiment_mean_predict = 'Opposed') AS count_opposed,
            COUNT(*) FILTER(classifications.sentiment_mean_predict = 'Support') AS count_support,

        FROM dupe_groups
        LEFT JOIN dupe_group_comments
            ON dupe_groups.id = dupe_group_comments.group_id
        LEFT JOIN comments
            ON dupe_group_comments.comment_id = comments.id
        LEFT JOIN classifications
            ON comments.id = classifications.comment_id
        GROUP BY dupe_groups.id
    )
    SELECT
        *,
        CASE
            WHEN count_opposed > count_support THEN 'Opposed'
            WHEN count_opposed < count_support THEN 'Support'
            ELSE 'Ambiguous'
        END AS majority_sentiment,
    FROM group_counts
"""

dupe_group_sentiments = duckdb.query(query).to_df()

In [52]:
dupe_groups = pd.merge(dupe_groups, dupe_group_sentiments, left_on="id", right_on="id")

dupe_groups.head()

,id,sample_comment_text,count_members_x,count_members_less_than_2_minute_interval,less_than_2_min_ratio,is_bot,count_members_y,count_opposed,count_support,majority_sentiment
0,0,Leave Trans Kids Alone This will hurt kids. ...,4419,4417,0.999547,True,4419,4419,0,Opposed
1,1,Strongly Oppose This will hurt kids. Don't b...,4227,4222,0.998817,True,4227,4227,0,Opposed
2,2,Youngkin Hates This will hurt kids. Don't be...,2740,2738,0.999270,True,2740,2740,0,Opposed
3,3,I fully oppose this. I fully oppose this.,1540,1459,0.947403,True,1540,1540,0,Opposed
4,4,Glenn Youngkin Is A Bot This will hurt kids....,1363,1362,0.999266,True,1363,1363,0,Opposed


In [56]:
dupe_groups.to_csv("data/model/dupe_groups.csv", index=False)

In [73]:
query = """
    WITH comment_dupes AS (
        SELECT
            comments.id,
            COUNT(*) FILTER(dupe_group_comments.comment_id IS NOT NULL) > 0 AS is_dupe,
        FROM comments
        LEFT JOIN dupe_group_comments
            ON comments.id = dupe_group_comments.comment_id
        GROUP BY comments.id
    )
    SELECT
        comments.*,
        comment_dupes.is_dupe,
    FROM comments
    LEFT JOIN comment_dupes
        ON comments.id = comment_dupes.id
"""

comments = duckdb.query(query).to_df()

In [74]:
comments[comments["is_dupe"]].head()

,id,url,label,title,content,html_raw,created_at,author,full_text,is_dupe
14,203027,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Protect kids,Strongly oppose.,<!DOCTYPE html> <p>Strongly oppose.</p>,2022-10-26 23:59:00,Anonymous (203027),Protect kids Strongly oppose.,True
23,203013,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Trans rights are human rights,I am writing in opposition to the ill-advise...,<!DOCTYPE html> <p>I am writing in oppositio...,2022-10-26 23:58:00,Shawn,Trans rights are human rights I am writing i...,True
24,203014,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Trans rights are human rights,I am writing in opposition to the ill-advise...,<!DOCTYPE html> <p>I am writing in oppositio...,2022-10-26 23:58:00,Shawn,Trans rights are human rights I am writing i...,True
27,203008,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Trans rights,Trans. Rights.,<!DOCTYPE html> <p>Trans. Rights. </p>,2022-10-26 23:58:00,Smith,Trans rights Trans. Rights.,True
52,202978,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,strongly oppose,Trans rights are human rights!!!!,<!DOCTYPE html> <p>Trans rights are human ri...,2022-10-26 23:54:00,Anonymous (202978),strongly oppose Trans rights are human right...,True


In [83]:
comments.to_csv("data/model/dupe_groups.csv", index=False)

In [78]:
query = """
    SELECT *
    FROM comments
    LEFT JOIN classifications
        ON comments.id = classifications.comment_id
"""

classified_comments = duckdb.query(query).to_df()

classified_comments.head()

,id,url,label,title,content,html_raw,created_at,author,full_text,is_dupe,comment_id,oppose_mean_similarity,support_mean_similarity,ambig_mean_similarity,sentiment_mean_predict,match_title,match_key,match_mean_sentiment
0,203047,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Transgender,I am strongly opposed to any law that would ...,<!DOCTYPE html> <p>I am strongly opposed to ...,2022-10-27 03:07:00,Frank McCullough,Transgender I am strongly opposed to any law...,False,203047,0.433871,0.340971,0.225158,Opposed,Strongly Oppose,https://townhall.virginia.gov/L/viewcomments.c...,Opposed
1,203046,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,"OPPOSE - this will cause actual, real, death o...","These kids are valid in their identities, ta...",<!DOCTYPE html> <p>These kids are valid in t...,2022-10-27 01:55:00,an APS student,"OPPOSE - this will cause actual, real, death o...",False,203046,0.606510,0.261078,0.132412,Opposed,Strongly oppose,https://townhall.virginia.gov/L/viewcomments.c...,Opposed
2,203045,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,"Consequences of Youngkin’s Policy Dire, Immediate",Gov. Youngkin's model policies regarding tra...,<!DOCTYPE html> <p>Gov. Youngkin's model pol...,2022-10-27 00:06:00,"Martha Molinaro, George Mason University","Consequences of Youngkin’s Policy Dire, Immedi...",False,203045,0.449680,0.320218,0.230103,Opposed,Model Policies for the Treatment of Transgende...,https://townhall.virginia.gov/L/viewcomments.c...,Opposed
3,203044,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Strongly Oppose,I strongly oppose the non-evidence based pol...,<!DOCTYPE html> <p>I <strong>strongly oppose...,2022-10-27 00:04:00,Anonymous (203044),Strongly Oppose I strongly oppose the non-ev...,False,203044,0.548306,0.292629,0.159066,Opposed,Strongly oppose,https://townhall.virginia.gov/L/viewcomments.c...,Opposed
4,203041,https://townhall.virginia.gov/L/viewcomments.c...,not_applicable,Opposed,"""Parents"" put a private equity sellout in ch...","<!DOCTYPE html> <p>""Parents"" put a private e...",2022-10-27 00:02:00,Anonymous (203041),"Opposed ""Parents"" put a private equity sello...",False,203041,0.396713,0.412091,0.191196,Support,I fully support,https://townhall.virginia.gov/L/viewcomments.c...,Support


In [80]:
classified_comments.groupby("sentiment_mean_predict", dropna=False).size()

sentiment_mean_predict
Ambiguous     2054
Opposed      44963
Support      24079
NaN            181
dtype: int64

In [82]:
classified_comments.groupby(
    ["sentiment_mean_predict", "is_dupe"],
    dropna=False,
).size().unstack()

is_dupe,False,True
sentiment_mean_predict,,
Ambiguous,1785,269
Opposed,17902,27061
Support,13002,11077
NaN,172,9
